In [1]:
import tensorflow as tf
import numpy as np

G:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


**_Exercise: Build a DNN with five hidden layers of 100 neurons each, He initialization, and the ELU activation function._**

In [2]:
def reset_graph():
    tf.reset_default_graph()

In [3]:
he_init = tf.variance_scaling_initializer()
def dnn(inputs, n_hidden_layers=5, n_neurons=100, activation=tf.nn.elu, initializer=he_init, name=None):
    with tf.variable_scope(name, "dnn"):
        for layer in range(n_hidden_layers):
            inputs = tf.layers.dense(inputs, n_neurons, activation=activation, 
                                     kernel_initializer=initializer, name="hidden%d" % (layer + 1))
        return inputs

In [4]:
n_inputs = 28 * 28  # MNIST
n_outputs = 5

reset_graph()

X = tf.placeholder(dtype=tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(dtype=tf.int32, shape=(None), name="y")

dnn_outputs = dnn(X)
logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=he_init, name="logits")
Y_proba = tf.nn.softmax(logits=logits, name="Y_proba")

**_Exercise: Using Adam optimization and early stopping, try training it on MNIST but only on digits 0 to 4._**

In [16]:
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split

In [6]:
data = fetch_mldata("MNIST original")
data

{'DESCR': 'mldata.org dataset: mnist-original',
 'COL_NAMES': ['label', 'data'],
 'target': array([0., 0., 0., ..., 9., 9., 9.]),
 'data': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)}

In [7]:
X_all = data["data"]
y_all = data["target"]
X_all.shape, y_all.shape

((70000, 784), (70000,))

In [8]:
np.random.permutation

<function RandomState.permutation>

But we must shuffle these data. We can use ``np.random.permutation`` for shuffle index creation and then extract the data we need for training and test datasets. Or we just can use ``tf.keras.mnist.load_data()`` to download already shuffled data

In [9]:
(X_train_all, y_train_all), (X_test_all, y_test_all) = tf.keras.datasets.mnist.load_data()
X_train_all.shape, y_train_all.shape, X_test_all.shape, y_test_all.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [10]:
X_train_all = X_train_all.reshape(-1, 28 * 28) / 255.0  # reshape and scale the data
X_test_all = X_test_all.reshape(-1, 28 * 28) / 255.0
y_train_all = y_train_all.astype(np.int32)
y_test_all = y_test_all.astype(np.int32)

As we can see our targets are already shuffled:

In [11]:
y_train_all[:10]

array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4])

Extract only targets from 0 to 4 for trainig and test datasets:

In [12]:
X_train = X_train_all[y_train_all < 5]  # zero to four
y_train = y_train_all[y_train_all < 5]
X_test = X_test_all[y_test_all < 5]
y_test = y_test_all[y_test_all < 5]

Let's check that the data is correct:

In [13]:
print("Digits in train: ", np.unique(y_train))
print("Digits in test: ", np.unique(y_test))

Digits in train:  [0 1 2 3 4]
Digits in test:  [0 1 2 3 4]


In [14]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((30596, 784), (30596,), (5139, 784), (5139,))

In [17]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=.20)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((24476, 784), (24476,), (6120, 784), (6120,))

In [16]:
for op in tf.get_default_graph().get_operations():
    print(op.name)

X
y
dnn/hidden1/kernel/Initializer/truncated_normal/shape
dnn/hidden1/kernel/Initializer/truncated_normal/mean
dnn/hidden1/kernel/Initializer/truncated_normal/stddev
dnn/hidden1/kernel/Initializer/truncated_normal/TruncatedNormal
dnn/hidden1/kernel/Initializer/truncated_normal/mul
dnn/hidden1/kernel/Initializer/truncated_normal
dnn/hidden1/kernel
dnn/hidden1/kernel/Assign
dnn/hidden1/kernel/read
dnn/hidden1/bias/Initializer/zeros
dnn/hidden1/bias
dnn/hidden1/bias/Assign
dnn/hidden1/bias/read
dnn/hidden1/MatMul
dnn/hidden1/BiasAdd
dnn/hidden1/Elu
dnn/hidden2/kernel/Initializer/truncated_normal/shape
dnn/hidden2/kernel/Initializer/truncated_normal/mean
dnn/hidden2/kernel/Initializer/truncated_normal/stddev
dnn/hidden2/kernel/Initializer/truncated_normal/TruncatedNormal
dnn/hidden2/kernel/Initializer/truncated_normal/mul
dnn/hidden2/kernel/Initializer/truncated_normal
dnn/hidden2/kernel
dnn/hidden2/kernel/Assign
dnn/hidden2/kernel/read
dnn/hidden2/bias/Initializer/zeros
dnn/hidden2/bias
d

Based on our graph operations we have to create nodes for the loss function, training operation and optionally for evaluation. Also the init node for initialization and the saver node for saving.

In [17]:
learning_rate = 0.01

In [18]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, name="Adam")
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()
mse_summary = tf.summary.scalar("MSE", loss)
file_writer = tf.summary.FileWriter("../tf_logs/mnist_model", tf.get_default_graph())

Our graph is ready. Let's go to the training phase. Remember that we have to implement early stopping.

In [19]:
n_epochs = 1000
batch_size = 20
max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    n_batches = len(X_train) // batch_size
    init.run()
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train))
        for batch_num, batch_idx in enumerate(np.array_split(rnd_idx, n_batches)):
            X_batch, y_batch = X_train[batch_idx], y_train[batch_idx]
            if batch_num % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_num
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_val, y: y_val})
        acc_train = sess.run(accuracy, feed_dict={X: X_train, y: y_train})
        if loss_val < best_loss:
            save_path = saver.save(sess, "../models/mnist_model/mnist_model_0_to_4.ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("Epoch {}:\tValidation loss: {:.3f}\tBest loss: {:.3f}\tAccuracy rate (val): {:.2f}%\tAccuracy rate (train): {:.2f}%".format(epoch, 
                                                                                                   loss_val, 
                                                                                                   best_loss, 
                                                                                                   acc_val * 100, acc_train * 100))

with tf.Session() as sess:
    saver.restore(sess, save_path)
    acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

Epoch 0:	Validation loss: 0.281	Best loss: 0.281	Accuracy rate (val): 95.13%	Accuracy rate (train): 95.10%
Epoch 1:	Validation loss: 0.094	Best loss: 0.094	Accuracy rate (val): 97.73%	Accuracy rate (train): 97.86%
Epoch 2:	Validation loss: 0.105	Best loss: 0.094	Accuracy rate (val): 97.75%	Accuracy rate (train): 97.82%
Epoch 3:	Validation loss: 0.130	Best loss: 0.094	Accuracy rate (val): 96.94%	Accuracy rate (train): 97.40%
Epoch 4:	Validation loss: 0.094	Best loss: 0.094	Accuracy rate (val): 97.17%	Accuracy rate (train): 97.58%
Epoch 5:	Validation loss: 0.662	Best loss: 0.094	Accuracy rate (val): 78.81%	Accuracy rate (train): 79.05%
Epoch 6:	Validation loss: 0.213	Best loss: 0.094	Accuracy rate (val): 94.98%	Accuracy rate (train): 95.42%
Epoch 7:	Validation loss: 0.131	Best loss: 0.094	Accuracy rate (val): 97.14%	Accuracy rate (train): 97.59%
Epoch 8:	Validation loss: 0.199	Best loss: 0.094	Accuracy rate (val): 95.67%	Accuracy rate (train): 96.06%
Epoch 9:	Validation loss: 0.261	Best 

**_Excercise: Use grid search to find the best parameters for the model_**

In [1]:
import tensorflow as tf
import numpy as np

G:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.exceptions import NotFittedError

Let's create our own Estimator to use it in ``RandomizedSearchCV`` from sklearn library:

In [3]:
he_init = tf.variance_scaling_initializer()

In [45]:
class DNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_hidden_layers=5, n_neurons=100, activation=tf.nn.elu, 
                 optimizer_class=tf.train.AdamOptimizer, initializer=he_init, learning_rate=0.01, 
                 batch_size=20, batch_normalization_momentum=None, dropout_rate=None, random_state=None):
        self.n_hidden_layers = n_hidden_layers
        self.n_neurons = n_neurons
        self.activation = activation
        self.optimizer_class = optimizer_class
        self.initializer = initializer
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.batch_normalization_momentum = batch_normalization_momentum
        self.dropout_rate = dropout_rate
        self.random_state = random_state
        self._session = None
            
    def _dnn(self, inputs):
        for layer in range(self.n_hidden_layers):
            if self.dropout_rate:
                inputs = tf.layers.dropout(inputs, rate=self.dropout_rate, training=self._training)
            inputs = tf.layers.dense(inputs, self.n_neurons, kernel_initializer=self.initializer, 
                                     name="hidden%d" % (layer + 1))
            if self.batch_normalization_momentum:
                inputs = tf.layers.batch_normalization(inputs, momentum=self.batch_normalization_momentum, 
                                                       training=self._training)
            inputs = self.activation(inputs, name="hidden%d_out" % (layer + 1))
        return inputs
    
    def _build_graph(self, n_inputs, n_outputs):
        if self.random_state is not None:
            tf.set_random_seed(self.random_state)
            np.random.seed(self.random_state)
        
        if self.dropout_rate or self.batch_normalization_momentum:
            self._training = tf.placeholder_with_default(False, shape=(), name="training")
        else:
            self._training = None
        
        X = tf.placeholder(dtype=tf.float32, shape=(None, n_inputs), name="X")
        y = tf.placeholder(dtype=tf.int32, shape=(None), name="y")
        
        dnn_outputs = self._dnn(X)
        
        logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=self.initializer, name="logits")
        Y_proba = tf.nn.softmax(logits=logits, name="Y_proba")
        
        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
        loss = tf.reduce_mean(xentropy, name="loss")
        
        optimizer = self.optimizer_class(learning_rate=self.learning_rate)
        training_op = optimizer.minimize(loss)
        
        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
        
        init = tf.global_variables_initializer()
        saver = tf.train.Saver()
        
        self._X, self._y = X, y
        self._logits = logits
        self._Y_proba, self._loss = Y_proba, loss
        self._training_op, self._accuracy = training_op, accuracy
        self._init, self._saver = init, saver
    
    def close_session(self):
        if self._session:
            self._session.close()
    
    def fit(self, X, y, n_epochs=100, X_valid=None, y_valid=None):
        self.close_session()
        
        n_inputs = X.shape[1]
        self.classes_ = np.unique(y)
        n_outputs = len(self.classes_)
        
        # Translate the labels vector to a vector of sorted class indices, containing
        # integers from 0 to n_outputs - 1.
        # For example, if y is equal to [8, 8, 9, 5, 7, 6, 6, 6], then the sorted class
        # labels (self.classes_) will be equal to [5, 6, 7, 8, 9], and the labels vector
        # will be translated to [3, 3, 4, 0, 2, 1, 1, 1]
        self.class_to_index_ = {label: index
                                for index, label in enumerate(self.classes_)}
        y = np.array([self.class_to_index_[label]
                      for label in y], dtype=np.int32)
        
        self._graph = tf.Graph()
        with self._graph.as_default():
            self._build_graph(n_inputs, n_outputs)
            # for batch normalization
            extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        
        # for early stopping
        max_checks_without_progress = 20
        checks_without_progress = 0
        best_loss = np.infty
        best_params = None
        
        self._session = tf.Session(graph=self._graph)
        with self._session.as_default() as sess:
            self._init.run()
            for epoch in range(n_epochs):
                rnd_idx = np.random.permutation(len(X))
                for batch_idx in np.array_split(rnd_idx, len(X) // self.batch_size):
                    X_batch, y_batch = X[batch_idx], y[batch_idx]
                    feed_dict = {self._X: X_batch, self._y: y_batch}
                    if self._training:
                        feed_dict[self._training] = True
                    sess.run(self._training_op, feed_dict=feed_dict)
                    if extra_update_ops:
                        sess.run(extra_update_ops, feed_dict=feed_dict)
                if X_valid is not None and y_valid is not None:
                    loss_val, acc_val = sess.run([self._loss, self._accuracy], 
                                                 feed_dict={self._X: X_valid, self._y: y_valid})
                if loss_val < best_loss:
                    best_loss = loss_val
                    save_path = self._saver.save(sess, "../models/DNNClassifier_model/best_params_model.ckpt")
                    checks_without_progress = 0
                else:
                    checks_without_progress += 1
                print("Epoch {:2}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(epoch, best_loss, acc_val * 100))
                if checks_without_progress > max_checks_without_progress:
                    print("Early stopping!")
                    break
        if save_path:
            with self._session.as_default() as sess:
                self._saver.restore(sess, save_path)
            return self
    
    def predict_proba(self, X):
        if not self._session:
            raise NotFittedError("This %s instance is not fitted yet" % self.__class__.__name__)
        with self._session.as_default() as sess:
            return self._Y_proba.eval(feed_dict={self._X: X})
                
    def predict(self, X):
        class_indices = np.argmax(self.predict_proba(X), axis=1)
        return np.array([self.classes_[class_index] for class_index in class_indices], dtype=np.int32)
    
    def save(self, path):
        self._saver.save(self._session, path)

In [47]:
my_dnn_classifier = DNNClassifier(random_state=42)

In [48]:
my_dnn_classifier.fit(X_train, y_train, X_valid=X_val, y_valid=y_val)

Epoch  0	Best loss: 0.126775	Accuracy: 97.14%
Epoch  1	Best loss: 0.126775	Accuracy: 60.11%
Epoch  2	Best loss: 0.126775	Accuracy: 96.11%
Epoch  3	Best loss: 0.126775	Accuracy: 96.88%
Epoch  4	Best loss: 0.126775	Accuracy: 96.86%
Epoch  5	Best loss: 0.126775	Accuracy: 95.46%
Epoch  6	Best loss: 0.126775	Accuracy: 84.82%
Epoch  7	Best loss: 0.126775	Accuracy: 97.04%
Epoch  8	Best loss: 0.126775	Accuracy: 96.65%
Epoch  9	Best loss: 0.126775	Accuracy: 97.42%
Epoch 10	Best loss: 0.126775	Accuracy: 95.10%
Epoch 11	Best loss: 0.122759	Accuracy: 96.94%
Epoch 12	Best loss: 0.122759	Accuracy: 97.76%
Epoch 13	Best loss: 0.122759	Accuracy: 97.30%
Epoch 14	Best loss: 0.122759	Accuracy: 84.92%
Epoch 15	Best loss: 0.122759	Accuracy: 96.52%
Epoch 16	Best loss: 0.122759	Accuracy: 96.91%
Epoch 17	Best loss: 0.122759	Accuracy: 30.60%
Epoch 18	Best loss: 0.122759	Accuracy: 22.27%
Epoch 19	Best loss: 0.122759	Accuracy: 19.41%
Epoch 20	Best loss: 0.122759	Accuracy: 19.41%
Epoch 21	Best loss: 0.122759	Accur

DNNClassifier(activation=<function elu at 0x0000027517ACFB70>,
       batch_normalization_momentum=None, batch_size=20, dropout_rate=None,
       initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x000002751C4E1C50>,
       learning_rate=0.01, n_hidden_layers=5, n_neurons=100,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42)

It works - fine

In [49]:
test_idx = np.random.randint(len(y_test), size=10)
test_X, test_y = X_test[test_idx], y_test[test_idx]
print("Original targets: ", test_y)
predictions = my_dnn_classifier.predict(test_X)
print("Predicted targets: ", predictions)

Original targets:  [2 3 3 0 0 0 2 1 3 2]
Predicted targets:  [2 3 3 0 0 3 2 1 3 2]


In [50]:
from sklearn.model_selection import RandomizedSearchCV

In [51]:
def leaky_relu(alpha=0.01):
    def parameterized_leaky_relu(z, name=None):
        return tf.maximum(z * alpha, z, name=name)
    return parameterized_leaky_relu

param_grid = {'n_neurons': [10, 30, 50, 70, 100, 120, 140, 160], 
              'batch_size': [20, 50, 100, 500], 
              'learning_rate': [0.01, 0.02, 0.05, 0.1], 
              'activation': [tf.nn.elu, tf.nn.relu, leaky_relu(alpha=0.01), leaky_relu(alpha=0.1)]}

In [52]:
rnd_search = RandomizedSearchCV(DNNClassifier(random_state=42), param_grid, n_iter=50, verbose=2, random_state=42)
rnd_search.fit(X_train, y_train, X_valid=X_val, y_valid=y_val, n_epochs=1000)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] n_neurons=10, learning_rate=0.05, batch_size=50, activation=<function leaky_relu.<locals>.parameterized_leaky_relu at 0x000002753BFAD400> 
Epoch  0	Best loss: 0.195989	Accuracy: 94.36%
Epoch  1	Best loss: 0.195989	Accuracy: 93.50%
Epoch  2	Best loss: 0.195989	Accuracy: 46.70%
Epoch  3	Best loss: 0.195989	Accuracy: 94.64%
Epoch  4	Best loss: 0.195989	Accuracy: 94.30%
Epoch  5	Best loss: 0.195989	Accuracy: 94.71%
Epoch  6	Best loss: 0.195989	Accuracy: 91.90%
Epoch  7	Best loss: 0.190199	Accuracy: 95.05%
Epoch  8	Best loss: 0.190199	Accuracy: 95.07%
Epoch  9	Best loss: 0.172477	Accuracy: 94.92%
Epoch 10	Best loss: 0.172477	Accuracy: 94.84%
Epoch 11	Best loss: 0.172477	Accuracy: 92.53%
Epoch 12	Best loss: 0.172477	Accuracy: 80.65%
Epoch 13	Best loss: 0.172477	Accuracy: 93.09%
Epoch 14	Best loss: 0.172477	Accuracy: 93.87%
Epoch 15	Best loss: 0.172477	Accuracy: 87.99%
Epoch 16	Best loss: 0.172477	Accuracy: 94.23%
Epoch 17	Bes

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   25.1s remaining:    0.0s


[CV] n_neurons=10, learning_rate=0.05, batch_size=50, activation=<function leaky_relu.<locals>.parameterized_leaky_relu at 0x000002753BFAD400> 
Epoch  0	Best loss: 0.260526	Accuracy: 93.77%
Epoch  1	Best loss: 0.260526	Accuracy: 77.30%
Epoch  2	Best loss: 0.235550	Accuracy: 93.33%
Epoch  3	Best loss: 0.235550	Accuracy: 92.94%
Epoch  4	Best loss: 0.180496	Accuracy: 95.83%
Epoch  5	Best loss: 0.170813	Accuracy: 95.08%
Epoch  6	Best loss: 0.170813	Accuracy: 94.18%
Epoch  7	Best loss: 0.170813	Accuracy: 92.88%
Epoch  8	Best loss: 0.170813	Accuracy: 94.98%
Epoch  9	Best loss: 0.159770	Accuracy: 95.96%
Epoch 10	Best loss: 0.159770	Accuracy: 61.65%
Epoch 11	Best loss: 0.159770	Accuracy: 92.09%
Epoch 12	Best loss: 0.159770	Accuracy: 94.95%
Epoch 13	Best loss: 0.159770	Accuracy: 94.56%
Epoch 14	Best loss: 0.159770	Accuracy: 95.25%
Epoch 15	Best loss: 0.159770	Accuracy: 95.88%
Epoch 16	Best loss: 0.159770	Accuracy: 95.33%
Epoch 17	Best loss: 0.159770	Accuracy: 95.41%
Epoch 18	Best loss: 0.159770

Epoch  6	Best loss: 0.083461	Accuracy: 97.29%
Epoch  7	Best loss: 0.083461	Accuracy: 97.61%
Epoch  8	Best loss: 0.083461	Accuracy: 97.96%
Epoch  9	Best loss: 0.083461	Accuracy: 97.97%
Epoch 10	Best loss: 0.083461	Accuracy: 97.12%
Epoch 11	Best loss: 0.083461	Accuracy: 98.20%
Epoch 12	Best loss: 0.083461	Accuracy: 98.10%
Epoch 13	Best loss: 0.083461	Accuracy: 98.07%
Epoch 14	Best loss: 0.083461	Accuracy: 98.19%
Epoch 15	Best loss: 0.079644	Accuracy: 97.97%
Epoch 16	Best loss: 0.079644	Accuracy: 97.97%
Epoch 17	Best loss: 0.079644	Accuracy: 97.75%
Epoch 18	Best loss: 0.079644	Accuracy: 98.33%
Epoch 19	Best loss: 0.079644	Accuracy: 98.17%
Epoch 20	Best loss: 0.079644	Accuracy: 98.28%
Epoch 21	Best loss: 0.079644	Accuracy: 98.15%
Epoch 22	Best loss: 0.079644	Accuracy: 98.06%
Epoch 23	Best loss: 0.079644	Accuracy: 98.37%
Epoch 24	Best loss: 0.079644	Accuracy: 97.48%
Epoch 25	Best loss: 0.079644	Accuracy: 98.32%
Epoch 26	Best loss: 0.079644	Accuracy: 98.45%
Epoch 27	Best loss: 0.079644	Accur

Epoch  3	Best loss: 1.616909	Accuracy: 22.27%
Epoch  4	Best loss: 1.616909	Accuracy: 19.41%
Epoch  5	Best loss: 1.616909	Accuracy: 19.36%
Epoch  6	Best loss: 1.616764	Accuracy: 22.27%
Epoch  7	Best loss: 1.616764	Accuracy: 22.27%
Epoch  8	Best loss: 1.609797	Accuracy: 22.27%
Epoch  9	Best loss: 1.609797	Accuracy: 19.36%
Epoch 10	Best loss: 1.609797	Accuracy: 20.31%
Epoch 11	Best loss: 1.609797	Accuracy: 22.27%
Epoch 12	Best loss: 1.609797	Accuracy: 22.27%
Epoch 13	Best loss: 1.609797	Accuracy: 19.36%
Epoch 14	Best loss: 1.609797	Accuracy: 20.31%
Epoch 15	Best loss: 1.609797	Accuracy: 20.31%
Epoch 16	Best loss: 1.609797	Accuracy: 19.41%
Epoch 17	Best loss: 1.609797	Accuracy: 20.31%
Epoch 18	Best loss: 1.609797	Accuracy: 22.27%
Epoch 19	Best loss: 1.609797	Accuracy: 18.64%
Epoch 20	Best loss: 1.609797	Accuracy: 19.41%
Epoch 21	Best loss: 1.609797	Accuracy: 20.31%
Epoch 22	Best loss: 1.608680	Accuracy: 22.27%
Epoch 23	Best loss: 1.608680	Accuracy: 22.27%
Epoch 24	Best loss: 1.608680	Accur

...

Epoch 44	Best loss: 3.880608	Accuracy: 95.46%
Epoch 45	Best loss: 3.880608	Accuracy: 95.34%
Epoch 46	Best loss: 3.880608	Accuracy: 95.23%
Epoch 47	Best loss: 3.880608	Accuracy: 95.83%
Epoch 48	Best loss: 3.880608	Accuracy: 95.11%
Epoch 49	Best loss: 3.880608	Accuracy: 90.85%
Epoch 50	Best loss: 3.880608	Accuracy: 94.84%
Epoch 51	Best loss: 3.880608	Accuracy: 95.16%
Epoch 52	Best loss: 3.880608	Accuracy: 95.83%
Epoch 53	Best loss: 3.880608	Accuracy: 94.75%
Epoch 54	Best loss: 3.880608	Accuracy: 89.22%
Epoch 55	Best loss: 3.880608	Accuracy: 92.43%
Epoch 56	Best loss: 3.880608	Accuracy: 92.89%
Epoch 57	Best loss: 3.880608	Accuracy: 96.19%
Epoch 58	Best loss: 3.880608	Accuracy: 94.89%
Early stopping!
INFO:tensorflow:Restoring parameters from ../models/DNNClassifier_model/best_params_model.ckpt
[CV]  n_neurons=120, learning_rate=0.05, batch_size=500, activation=<function leaky_relu.<locals>.parameterized_leaky_relu at 0x000002753BFAD598>, total=  29.7s
[CV] n_neurons=120, learning_rate=0.1,

Epoch  0	Best loss: 16292.290039	Accuracy: 20.31%
Epoch  1	Best loss: 27.077560	Accuracy: 76.21%
Epoch  2	Best loss: 4.786472	Accuracy: 86.80%
Epoch  3	Best loss: 3.408092	Accuracy: 88.45%
Epoch  4	Best loss: 3.408092	Accuracy: 84.75%
Epoch  5	Best loss: 2.289610	Accuracy: 91.00%
Epoch  6	Best loss: 2.289610	Accuracy: 89.64%
Epoch  7	Best loss: 2.289610	Accuracy: 90.15%
Epoch  8	Best loss: 1.655310	Accuracy: 91.80%
Epoch  9	Best loss: 1.655310	Accuracy: 91.36%
Epoch 10	Best loss: 1.655310	Accuracy: 85.20%
Epoch 11	Best loss: 1.655310	Accuracy: 90.70%
Epoch 12	Best loss: 1.655310	Accuracy: 85.00%
Epoch 13	Best loss: 1.655310	Accuracy: 93.99%
Epoch 14	Best loss: 1.655310	Accuracy: 86.90%
Epoch 15	Best loss: 1.655310	Accuracy: 90.67%
Epoch 16	Best loss: 1.655310	Accuracy: 92.21%
Epoch 17	Best loss: 1.608626	Accuracy: 94.10%
Epoch 18	Best loss: 1.286017	Accuracy: 94.43%
Epoch 19	Best loss: 1.024147	Accuracy: 94.85%
Epoch 20	Best loss: 1.024147	Accuracy: 93.59%
Epoch 21	Best loss: 1.024147	

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 48.4min finished


Epoch  0	Best loss: 0.091197	Accuracy: 97.84%
Epoch  1	Best loss: 0.078268	Accuracy: 98.01%
Epoch  2	Best loss: 0.075918	Accuracy: 98.22%
Epoch  3	Best loss: 0.075918	Accuracy: 97.57%
Epoch  4	Best loss: 0.072194	Accuracy: 98.24%
Epoch  5	Best loss: 0.064982	Accuracy: 98.48%
Epoch  6	Best loss: 0.063975	Accuracy: 98.81%
Epoch  7	Best loss: 0.063975	Accuracy: 98.71%
Epoch  8	Best loss: 0.063975	Accuracy: 97.70%
Epoch  9	Best loss: 0.063975	Accuracy: 98.40%
Epoch 10	Best loss: 0.063975	Accuracy: 98.68%
Epoch 11	Best loss: 0.057608	Accuracy: 98.73%
Epoch 12	Best loss: 0.057608	Accuracy: 98.56%
Epoch 13	Best loss: 0.057608	Accuracy: 97.61%
Epoch 14	Best loss: 0.057608	Accuracy: 98.45%
Epoch 15	Best loss: 0.057608	Accuracy: 98.61%
Epoch 16	Best loss: 0.057608	Accuracy: 98.45%
Epoch 17	Best loss: 0.057608	Accuracy: 98.56%
Epoch 18	Best loss: 0.057608	Accuracy: 98.43%
Epoch 19	Best loss: 0.057608	Accuracy: 98.40%
Epoch 20	Best loss: 0.057608	Accuracy: 98.45%
Epoch 21	Best loss: 0.057608	Accur

RandomizedSearchCV(cv=None, error_score='raise',
          estimator=DNNClassifier(activation=<function elu at 0x0000027517ACFB70>,
       batch_normalization_momentum=None, batch_size=20, dropout_rate=None,
       initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x000002751C4E1C50>,
       learning_rate=0.01, n_hidden_layers=5, n_neurons=100,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42),
          fit_params=None, iid=True, n_iter=50, n_jobs=1,
          param_distributions={'n_neurons': [10, 30, 50, 70, 100, 120, 140, 160], 'batch_size': [20, 50, 100, 500], 'learning_rate': [0.01, 0.02, 0.05, 0.1], 'activation': [<function elu at 0x0000027517ACFB70>, <function relu at 0x0000027517B00EA0>, <function leaky_relu.<locals>.parameterized_leaky_relu at 0x000002753BFAD400>, <function leaky_relu.<locals>.parameterized_leaky_relu at 0x000002753BFAD598>]},
          pre_dispatch='2*n_jobs', random_state=42, refit=Tr

In [53]:
rnd_search.best_params_

{'n_neurons': 70,
 'learning_rate': 0.01,
 'batch_size': 100,
 'activation': <function __main__.leaky_relu.<locals>.parameterized_leaky_relu(z, name=None)>}

In [55]:
print("Original targets: ", test_y)
predictions = rnd_search.predict(test_X)
print("Predicted targets: ", predictions)

Original targets:  [2 3 3 0 0 0 2 1 3 2]
Predicted targets:  [2 3 3 0 0 0 2 1 3 2]


In [58]:
from sklearn.metrics import accuracy_score
y_pred = rnd_search.predict(X_test)
print("Accuracy score: {}".format(accuracy_score(y_test, y_pred)))

Accuracy score: 0.9900758902510216


**Not bad**